In [1]:
# set up earth engine
import ee

#ee.Authenticate()

In [2]:
ee.Initialize()

In [5]:
# import prisons
prison_centroids = ee.FeatureCollection("projects/ee-ccmothes/assets/prison_centroids")


In [3]:
# define date range
startDate = "2012-01-01"
endDate = "2022-12-31"


In [21]:
# Define function to convert Kelvin to Celcius
def toCelciusDay(image):
  lst = image.select('LST_Day').multiply(0.02).subtract(273.15)
  overwrite = True
  result = image.addBands(lst, ['LST_Day'], overwrite)
  return result

In [22]:
# Quality mask; code adopted from https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
def bitwiseExtract(input, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)


#Let's extract all pixels from the input image where
#Bits 0-1 <= 1 (LST produced of both good and other quality)
#Bits 2-3 = 0 (Good data quality)
#Bits 4-5 Ignore, any value is ok
#Bits 6-7 = 0 (Average LST error ≤ 1K)
def applyQaMask(image):
  lstDay = image.select('LST_Day')
  qcDay = image.select('QC_Day')
  qaMask = bitwiseExtract(qcDay, 0, 1).lte(1)
  dataQualityMask = bitwiseExtract(qcDay, 2, 3).eq(0)
  lstErrorMask = bitwiseExtract(qcDay, 6, 7).lte(1)
  mask = qaMask.And(dataQualityMask).And(lstErrorMask)
  return lstDay.updateMask(mask)


In [23]:
#import MODIS
modisdata = ee.ImageCollection('MODIS/061/MYD21C1') \
  .filterDate(ee.Date(startDate),ee.Date(endDate)) \
  .filter(ee.Filter.calendarRange(6, 8,'month'))
  

modisdata.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'Count_Day',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 90]},
  {'id': 'Count_Night',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 90]},
  {'id': 'QC_Day',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 90]},
  {'id': 'QC_Night',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 90]},
  {'id': 'LST_Day',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 1310.7},
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 90]},
  {'id': 'LST_Nig

In [24]:
# Apply processing functions
lst_day_processed = modisdata.map(toCelciusDay).map(applyQaMask)

In [18]:
# Now calculate average summer day temperature across date range
summer_day_lst = lst_day_processed.select('LST_Day').median()

In [9]:
# calculate extreme heat days
def hotdays(image):
        hot = image.gt(35)
        return image.addBands(hot.rename('hotdays')
                              .set('system:time_start', image.get('system:time_start')))

In [31]:
# test calculating hot days for 2012
lst_hotdays = ee.ImageCollection(lst_day_processed.select('LST_Day')).map(hotdays)

lst_hotdays_2012 = ee.ImageCollection(lst_hotdays.select('hotdays')).sum().float()

In [11]:
import geemap

In [32]:
lst_hotdays_2012.getInfo()

{'type': 'Image',
 'bands': [{'id': 'hotdays',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [41]:
# map 2012 hot days

HotDaysVis = {'min':0, 'max':5, 'palette':['white','yellow','orange','red']}
Map = geemap.Map()
Map.addLayer(lst_hotdays_2012, HotDaysVis, "Hot Days", False)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [38]:

# Extract values at prison centroids
# sampled_points = summer_day_lst.sampleRegions(
#   collection=prison_centroids,
#   scale=1000
# )                         
    

In [32]:
# results = sampled_points.flatten()

In [33]:
# print(results)


ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.flatten",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Image.sampleRegions",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "projects/ee-ccmothes/assets/prison_centroids"
                  }
                }
              }
            },
            "image": {
              "functionInvocationValue": {
                "functionName": "reduce.median",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.map",
                      "arguments": {
                        "baseAlgorithm": {
                          "functionDefinition

In [35]:
# try using other method

#yearly = ee.ImageCollection('JRC/GSW1_0/YearlyHistory')

# function to map over the FeatureCollection
def mapfunc(feat):
  #get feature geometry
    geom = feat.geometry()
    # function to iterate over the yearly ImageCollection
    # the initial object for the iteration is the feature
    def addProp(img, f):
    # cast Feature
    newf = ee.Feature(f)
    #get date as string
    date = img.date().format()
    #extract the value (first) of 'waterClass' in the feature
    value = img.reduceRegion(ee.Reducer.first(), geom, 30).get('waterClass')
    #if the value is not null, set the values as a property of the feature. The name of the property will be the date
    return ee.Feature(ee.Algorithms.If(value,
                                       newf.set(date, ee.String(value)),
                                       newf.set(date, ee.String('No data'))))

  newfeat = ee.Feature(summer_day_lst.iterate(addProp, feat))
  return newfeat


newft = prison_centroids.map(mapfunc)

#Export
#Export.table.toDrive(newft,
#"export_Points",
#"export_Points",
#"export_Points");



{'type': 'Image',
 'bands': [{'id': 'LST_Day_1km',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -273.15,
    'max': 1037.5500000000002},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

Try using geemap to extract values to points

In [12]:
import geemap
import os

In [13]:
# set directory to save csv to
work_dir = os.path.expanduser('C:/Users/ccmothes/Desktop/NASA-prison-EJ/data/processed/')

In [15]:
#create empty csv file to save
out_csv = os.path.join(work_dir, 'modis_lst.csv')

#extract and save csv
geemap.extract_values_to_points(prison_centroids, summer_day_lst, out_csv, scale = 1000)

EEException: User memory limit exceeded.

Still exceeds memory limits. Will need to reduce @ polygons instead